In [1]:
import torch
import torch.nn as nn
import sys
import os

In [2]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [8]:
path = "./runs/5_32_500_1e-3_1e-10_Adam_0_25_3class"

In [9]:
decided_model = f"{path}/best-model.pt" #Changed to selected model

In [10]:
from model import ResNetScratch

In [11]:
model = ResNetScratch(out_dim=3)

In [12]:
model.load_state_dict(torch.load(decided_model))
model.to(device)
model.eval()

ResNetScratch(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
      

In [13]:
from data import create_splits
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report

In [14]:
train_set, val_set, test_set = create_splits(dataset_type="5class")
set(test_set.img_labels)

Image dir: 681, Image labels: 681
Image dir: 37, Image labels: 37
Image dir: 703, Image labels: 703


{0, 1, 2, 3, 4}

In [15]:
train_set, val_set, test_set = create_splits(dataset_type="5class")

#Conversion Part
conversion = lambda x: 1 if ((x>=1) & (x<4)) else 2 if (x==4) else 0
test_set.img_labels = [conversion(i) for i in test_set.img_labels]

Image dir: 683, Image labels: 683
Image dir: 35, Image labels: 35
Image dir: 703, Image labels: 703


In [16]:
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, num_workers=2)

In [17]:
np.unique(np.array(test_set.img_labels))

array([0, 1, 2])

In [18]:
def test(device, model, criterion, test_loader, savedir):
    cur_test_loss = 0
    test_tot, test_corr = 0, 0
    model.eval()

    all_preds, all_labels, all_indices = [], [], []
    with torch.no_grad():
        for images, labels, index in test_loader:
            images, labels = images.to(device), labels.to(device)
            outs = model(images)
            test_loss = criterion(outs, labels)
            cur_test_loss += test_loss.item()

            _, preds = torch.max(outs.data, 1)
            test_tot += labels.size(0)
            test_corr += (preds == labels).sum().item()

            preds = preds.cpu().tolist()
            labels = labels.cpu().tolist()
            index = index.tolist()

            all_preds.extend(preds)
            all_labels.extend(labels)
            all_indices.extend(index)
            
    test_loss = cur_test_loss / len(test_loader)
    test_acc = test_corr / test_tot

    print(f'Test loss: {test_loss:.3f}, Test accuracy: {test_acc:.3f}')

    #Print predictions by class
    confusion_matrix = pd.crosstab(np.array(all_preds), np.array(all_labels), rownames=["Predictions"], colnames=["Actuals"]) 
    print(confusion_matrix)

    #Accuracy Results
    classification_res = classification_report(all_labels, all_preds, digits=3)
    print(classification_res)


In [19]:
test(device, model, nn.CrossEntropyLoss(), test_loader, None)

Test loss: 0.059, Test accuracy: 0.980
Actuals        0    1   2
Predictions              
0            178    0   0
1              0  417  11
2              0    3  94
              precision    recall  f1-score   support

           0      1.000     1.000     1.000       178
           1      0.974     0.993     0.983       420
           2      0.969     0.895     0.931       105

    accuracy                          0.980       703
   macro avg      0.981     0.963     0.971       703
weighted avg      0.980     0.980     0.980       703

